<a href="https://colab.research.google.com/github/FranMoretti/FranMoretti/blob/main/Copia_de_05_lightfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install lightfm > /dev/null
# necesita python 3.10

In [2]:
import lightfm as lfm
from lightfm import data
from lightfm import cross_validation
from lightfm import evaluation
import pandas as pd
import numpy as np
import sqlite3

In [3]:
conn = sqlite3.connect('data.db')

df_int = pd.read_sql_query("SELECT * FROM interacciones", conn) #  WHERE rating >= 7
#EL WHERE LO PONE PORQUE ESTE MODELO LE DA LO MISMO UN 1 QUE UN 10, A AMBOS LE ASIGNA 1, ASIGNA CERO A LOS QUE NO TIENEN INTERACCIONES
#ESTE MODELO REEMPLAZA SURPRISE
#ES UNA ESPECIE DE FILTRADO COLABORATIVO
#puedo usar este modelo y guardarlo en pickle por ejemplo
#ACÁ ESTÁ HECHO PARA ITEMS, PUEDO HACERLO TAMBIÉN PARA USERS
df_int.head()

,id_lector,id_libro,fecha,rating
0,yura,-incluso-el-olvido,hace 8 años,6
1,albertofg1,1080-recetas-de-cocina,hace 4 años,9
2,trini,1080-recetas-de-cocina,hace 12 años,10
3,chicarela,11-4-suenos-luz,hace 4 años,9
4,jorge-knowles-5,13-99-euros,hace 10 años,9


In [4]:
ds = lfm.data.Dataset()
ds.fit(users=df_int["id_lector"].unique(), items=df_int["id_libro"].unique())
ds.interactions_shape()

(11221, 43740)

In [5]:
(interactions, weights) = ds.build_interactions(df_int[['id_lector', 'id_libro', 'rating']].itertuples(index=False)) # ojo con el orden
#interactions
#weights

In [87]:
#PRUEBA NO EXITOSA!!!!!!!!!!!!!!!!
from scipy.sparse import csr_matrix, coo_matrix
# Construir interacciones
(interactions, _) = ds.build_interactions(
    df_int[['id_lector', 'id_libro', 'rating']].itertuples(index=False)
)

row, col, _ = interactions.row, interactions.col, interactions.data
weights = np.where(df_int['rating'] >= 7, 2.0, 1.0)

# Crear matriz dispersa para los pesos en formato COO
weights_coo = coo_matrix((weights, (row, col)), shape=interactions.shape)

# Dividir en conjuntos de entrenamiento y prueba
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=42)
train_w, test_w = lfm.cross_validation.random_train_test_split(weights_coo, test_percentage=0.2, random_state=42)

# Asegurarse de que train_w esté alineado con train
train_w = coo_matrix(train_w)

# Crear y entrenar el modelo
model = lfm.LightFM(no_components=40, loss="bpr", learning_rate=0.01)
model.fit(train, sample_weight=train_w, epochs=10, num_threads=2)

# Evaluar precisión
train_precision = lfm.evaluation.precision_at_k(model, train, k=10, num_threads=2).mean()
test_precision = lfm.evaluation.precision_at_k(model, test, k=10, num_threads=2).mean()

print('Precisión en train: {}'.format(train_precision))
print('Precisión en test: {}'.format(test_precision))

Precisión en train: 0.09685785323381424
Precisión en test: 0.03339698165655136


In [22]:
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=42)
train_w, test_w = lfm.cross_validation.random_train_test_split(weights, test_percentage=0.2, random_state=42)
model = lfm.LightFM(no_components=30, loss="bpr",learning_rate = 0.02) # hiperparámetro
model.fit(train, sample_weight=train_w, epochs=10, num_threads=2)

In [23]:
train_precision = lfm.evaluation.precision_at_k(model, train, k=10, num_threads=2).mean()
test_precision  = lfm.evaluation.precision_at_k(model, test,  k=10, num_threads=2).mean()
print('Precision en train: {}'.format(train_precision))
print('Precision en test: {}'.format(test_precision))

Precision en train: 0.1532851606607437
Precision en test: 0.0496697723865509


In [19]:
#AGREGADO POR MI PARA PROBARRRRR!!!!!!!!!!!!!!!!
# Probar diferentes hiperparámetros
model_configs = [
    {'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.02},
    {'no_components': 30, 'loss': 'bpr', 'learning_rate': 0.02},
    {'no_components': 40, 'loss': 'bpr', 'learning_rate': 0.02}
]

# Asumo que ya tienes ds, interactions y weights definidos de tu código anterior
for config in model_configs:
    # Crear y entrenar el modelo con la configuración actual
    model = lfm.LightFM(**config)
    model.fit(train, sample_weight=train_w, epochs=10, num_threads=2)

    # Calcular métricas para esta configuración
    train_precision = evaluation.precision_at_k(model, train, k=10, num_threads=2).mean()
    test_precision = evaluation.precision_at_k(model, test, k=10, num_threads=2).mean()

    print(f"Configuración: {config}")
    print(f'Precision en train: {train_precision}')
    print(f'Precision en test: {test_precision}')

Configuración: {'no_components': 20, 'loss': 'bpr', 'learning_rate': 0.02}
Precision en train: 0.16185224056243896
Precision en test: 0.04936956241726875
Configuración: {'no_components': 30, 'loss': 'bpr', 'learning_rate': 0.02}
Precision en train: 0.1761338710784912
Precision en test: 0.05198138952255249
Configuración: {'no_components': 40, 'loss': 'bpr', 'learning_rate': 0.02}
Precision en train: 0.17375677824020386
Precision en test: 0.05042029917240143


In [18]:
#AGREGADO POR MI PARA PROBARRRRR!!!!!!!!!!!!!!!!
# Early stopping
from lightfm.evaluation import precision_at_k

def train_with_early_stopping(model, train, test, train_w, test_w, epochs=50, patience=5, num_threads=2):
    best_precision = 0
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.fit(train, sample_weight=train_w, epochs=1, num_threads=num_threads)

        # Evaluar en conjunto de prueba
        test_precision = precision_at_k(model, test, k=10, num_threads=num_threads).mean()
        print(f"Época {epoch + 1}: Precisión en test: {test_precision:.4f}")

        # Verificar si hay mejora
        if test_precision > best_precision:
            best_precision = test_precision
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        # Early stopping
        if epochs_without_improvement >= patience:
            print(f"Detenido en la época {epoch + 1} debido a falta de mejora.")
            break

    return model

# Llamar a la función de entrenamiento
model = lfm.LightFM(no_components=40, loss="bpr", learning_rate=0.02)
model = train_with_early_stopping(model, train, test, train_w, test_w, epochs=50, patience=5, num_threads=2)


Época 1: Precisión en test: 0.0398
Época 2: Precisión en test: 0.0417
Época 3: Precisión en test: 0.0418
Época 4: Precisión en test: 0.0395
Época 5: Precisión en test: 0.0385
Época 6: Precisión en test: 0.0412
Época 7: Precisión en test: 0.0409
Época 8: Precisión en test: 0.0405
Detenido en la época 8 debido a falta de mejora.


# Item features

In [ ]:
conn = sqlite3.connect('data.db')

df_items = pd.read_sql_query("SELECT * FROM libros", conn)
df_items = df_items[["id_libro", "autor", "genero"]]
df_items.head()

In [ ]:
ds = lfm.data.Dataset()
ds.fit(users=df_int["id_lector"].unique(), items=df_items["id_libro"].unique(), item_features=df_items["autor"].unique().tolist() + df_items["genero"].unique().tolist())
ds.interactions_shape()

In [ ]:
#df_items["autor"].unique().tolist() + df_items["genero"].unique().tolist()
len(df_items["autor"].unique().tolist() + df_items["genero"].unique().tolist())

In [ ]:
ifs = []
for _, row in df_items.iterrows():
    ifs.append( (row["id_libro"], [row["autor"], row["genero"]]) ) # { row["autor"]: 0.2, row["genero"]: 0.8 }

item_features = ds.build_item_features(ifs)
item_features

In [ ]:
#len(ifs)
# 24306

#(interactions, weights) = ds.build_interactions(df_int[['id_lector', 'id_libro', 'rating']].itertuples(index=False, name=None)) # ojo con el orden
#interactions
#weights

In [ ]:
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2, random_state=42)
train_w, test_w = lfm.cross_validation.random_train_test_split(weights, test_percentage=0.2, random_state=42)
model = lfm.LightFM(no_components=20, loss="warp")
model.fit(train, sample_weight=train_w, epochs=10, item_features=item_features, num_threads=2)

In [ ]:
train_precision = lfm.evaluation.precision_at_k(model, train, item_features=item_features, k=10, num_threads=2).mean()
test_precision  = lfm.evaluation.precision_at_k(model, test,  item_features=item_features, k=10, num_threads=2).mean()
print('Precision en train: {}'.format(train_precision))
print('Precision en test: {}'.format(test_precision))

# Predicción

In [ ]:
# consigo los mapeos creados por el modelo // el modelo a cada libro/user le crea un ID
user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = ds.mapping()
user_id_mapping
#item_id_mapping

In [ ]:
model = lfm.LightFM(no_components=20)
model.fit(interactions=interactions, sample_weight=weights, item_features=item_features, epochs=10, num_threads=2)

In [ ]:
#ESTA ES UNA PRUEBA CON UN USER Y DOS LIBROS, PERO YO DEBERÍA PASARLE TODOS LOS LIBROS QUE NO LEYÓ
#ESTE ES UNA PREDICCIÓN DE "ESTRELLITAS"
#ESTE ES UN MÉTODO MÁS RAPIDO
ID_LECTOR = "c3po"
IDS_LIBROS = ["yerma", "watchmen"] # todos los items

#user_id_mapping[ID_LECTOR]
#np.array([item_id_mapping[m] for m in IDS_LIBROS])
model.predict(user_id_mapping[ID_LECTOR], np.array([item_id_mapping[m] for m in IDS_LIBROS]), item_features=item_features, num_threads=2)

In [ ]:
# PARA VER
#ESTA ES UNA PREDICCIÓN DE RANKING // ESTE ES UN MÉTODO QUE SIRVE MÁS PARA POCOS USERS, PERO MAS LENTO SI HACES MUCHAS PREDICCIONES
model.predict_rank(interactions, item_features=item_features, num_threads=2, check_intersections=True)

In [ ]:
# https://making.lyst.com/lightfm/docs/lightfm.html